In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
path_to_src = os.path.join(rootdir, '_2_intermediate', 'data')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

In [3]:
out = pd.read_csv(os.path.join(path_to_src, '_distribution_similarity_y0_mct_bd.csv'), index_col=[0,1])

In [4]:
out

iso   w_all_m  r_all_m  r_all_m_byc  n_all_m_byc   w_all_t  \
bd     district                                                               
1880.0 356.0     EGY  0.000000      1.0          1.0         47.0       NaN   
       380.0     EGY  0.000000      2.0          2.0         47.0       NaN   
       484.0     EGY  0.000000      3.0          3.0         47.0       NaN   
       415.0     EGY  0.081218      8.0          4.0         47.0       NaN   
       413.0     EGY  0.456540      9.0          5.0         47.0       NaN   
...              ...       ...      ...          ...          ...       ...   
2010.0 2754.0    ZMB       NaN      NaN          NaN          NaN  2.438854   
       2725.0    ZMB       NaN      NaN          NaN          NaN  2.560976   
       2744.0    ZMB       NaN      NaN          NaN          NaN  2.618421   
       2723.0    ZMB       NaN      NaN          NaN          NaN  2.853994   
       2722.0    ZMB       NaN      NaN          NaN          NaN  3.209302   

                 r_all_t  r_all_t_byc  n_all_t_byc  w_1418_m  r_1418_m  \
bd     district                                                          
1880.0 356.0         NaN          NaN          NaN       NaN       NaN   
       380.0         NaN          NaN          NaN       NaN       NaN   
       484.0         NaN          NaN          NaN       NaN       NaN   
       415.0         NaN          NaN          NaN       NaN       NaN   
       413.0         NaN          NaN          NaN       NaN       NaN   
...                  ...          ...          ...       ...       ...   
2010.0 2754.0      374.0         47.0         51.0       NaN       NaN   
       2725.0      390.0         48.0         51.0       NaN       NaN   
       2744.0      400.0         49.0         51.0       NaN       NaN   
       2723.0      425.0         50.0         51.0       NaN       NaN   
       2722.0      451.0         51.0         51.0       NaN       NaN   

                 r_1418_m_byc  n_1418_m_byc  w_1418_t  r_1418_t  r_1418_t_byc  \
bd     district                                                                 
1880.0 356.0              NaN           NaN       NaN       NaN           NaN   
       380.0              NaN           NaN       NaN       NaN           NaN   
       484.0              NaN           NaN       NaN       NaN           NaN   
       415.0              NaN           NaN       NaN       NaN           NaN   
       413.0              NaN           NaN       NaN       NaN           NaN   
...                       ...           ...       ...       ...           ...   
2010.0 2754.0             NaN           NaN       NaN       NaN           NaN   
       2725.0             NaN           NaN       NaN       NaN           NaN   
       2744.0             NaN           NaN       NaN       NaN           NaN   
       2723.0             NaN           NaN       NaN       NaN           NaN   
       2722.0             NaN           NaN       NaN       NaN           NaN   

                 n_1418_t_byc  
bd     district                
1880.0 356.0              NaN  
       380.0              NaN  
       484.0              NaN  
       415.0              NaN  
       413.0              NaN  
...                       ...  
2010.0 2754.0             NaN  
       2725.0             NaN  
       2744.0             NaN  
       2723.0             NaN  
       2722.0             NaN  

[17144 rows x 17 columns]

In [5]:
def make_ridgeplot(df, birth_decade, comparison):
    
    plot_col = f'w_{comparison}'
    rank_col = f'r_{comparison}_byc'
    n_col = f'n_{comparison}_byc'
    pdf = out.loc[birth_decade][['iso', plot_col, rank_col, n_col]].dropna()
    pdf[f'{plot_col}_50'] = pdf[plot_col]
    pdf.loc[pdf[rank_col] >= pdf[n_col]/2, f'{plot_col}_50'] = np.nan
    pdf = pdf.sort_values('iso')
    
    
    g = sns.FacetGrid(pdf, row='iso', aspect=10, height=1)
    
    ISO = sorted(list(set(pdf.iso)))

    # then we add the densities kdeplots for each country
    g.map(sns.kdeplot, plot_col,
          bw_adjust=1, clip_on=False,
          fill=True, alpha=0.5, linewidth=1.5, color='b', label='full sample')
    g.map(sns.kdeplot, f'{plot_col}_50',
          bw_adjust=1, clip_on=False,
          fill=True, alpha=0.5, linewidth=1.5, color='r', label='50% closest districts')

    # we loop over the FacetGrid figure axes (g.axes.flat) and add the month as text with the right color
    # notice how ax.lines[-1].get_color() enables you to access the last line's color in each matplotlib.Axes
    for i, ax in enumerate(g.axes.flat):
        ax.set_ylabel(ISO[i], rotation='horizontal', size=16)
        if i==0:
            ax.legend(fontsize=16)
            
    g.fig.subplots_adjust(hspace=0.1)
    
    g.set_titles("")
    g.set(yticks=[])
    g.despine(left=True)
    plt.xlabel('Wasserstein distance Christian - Muslim', fontsize=16)
    plt.close(g.fig)
    return g.fig

In [6]:
birth_decade = 1980

comparison = 'all_m'
f = make_ridgeplot(out, birth_decade, comparison)
f.savefig(os.path.join(outdir, f'{comparison}_wasserstein_distances_bycountry.pdf'), bbox_inches='tight')
comparison = 'all_t'
f = make_ridgeplot(out, birth_decade, comparison)
f.savefig(os.path.join(outdir, f'{comparison}_wasserstein_distances_bycountry.pdf'), bbox_inches='tight')
comparison = '1418_m'
f = make_ridgeplot(out, birth_decade, comparison)
f.savefig(os.path.join(outdir, f'{comparison}_wasserstein_distances_bycountry.pdf'), bbox_inches='tight')
comparison = '1418_t'
f = make_ridgeplot(out, birth_decade, comparison)
f.savefig(os.path.join(outdir, f'{comparison}_wasserstein_distances_bycountry.pdf'), bbox_inches='tight')